# Calculate 13C chemical shifts from DCodes

Step 1: Store the DCodes in a Pandas DataFrame and then use them.

Step 2: Enter the molecule as SMILES and generate the codes.

Step 3: Formatted output.

Step 4: Transfer to Google Colab or similar.


In [2]:
# In der ersten CodeZelle erfolgt das Einlesen der Daten in das pandas Dataframe
# Dann muss es nicht jedes Mal neu erfolgen.

import pandas as pd
from rdkit import *
from rdkit.Chem import AllChem, Draw
from IPython.display import display, Image, HTML
import os, sys
import tempfile
import ipywidgets as widgets
from dcode import geometry, tools
from dcode.geometry import *
from dcode.tools import *
from dcode.calcshift import *


codefile="codes/v3_update_10_06_2025.csv"

# CSV-Datei einlesen
df = pd.read_csv(codefile)



In [4]:
####################################################################################
#                                                                                  #
#              Enter SMILES CODE here                                              #
#                                                                                  #
####################################################################################

smiles='[H]O[C@]1([H])[C@@]2([H])C([H])([H])C(=O)[C@]([H])(C2([H])[H])[C@]1([H])O[H]'


### Do not edit more things !!!!  ###



groesse=(450,450)   # definiert die Größe der Abbildung
current_dir = os.getcwd()

mol = Chem.MolFromSmiles(smiles) #Molecule from SMILES

molH=mol

if not Hda(mol): 
    molH=Chem.AddHs(mol)

molID=AllChem.EmbedMolecule(molH,randomSeed=435542)  # generate 3D coordinats 

# look for success

if (molID<0):
   sys.exit('Error in 3D coordinate generation. Program will exit.')

## optimize molecule
molOpt= ffoptimize(molH)

molCoded=DCodeName(molOpt)
## build distances
#molcoded is a molecule object with codes as an atomic property.
#Function call to store the code as an atomic property.
#As a second parameter, sort= Integer can be passed for the OZ of the atom type of interest.
#The default is 6 for carbon.
mol_mit_Code=DCodeMol(molCoded)
## DCode is now an atomic propertie
    

## build molecule
show_atom_number(molOpt, 'atomNote') #create numbering
mol_ohne_H = Chem.RemoveHs(molOpt)
AllChem.Compute2DCoords(mol_ohne_H)
img = Draw.MolToImage(mol_ohne_H, size=groesse)  # change size
## using tempfile library to create a temporary image to avoid problems updating the display
## molbild is a temporary name for the picture


with tempfile.NamedTemporaryFile(dir=os.getcwd(), suffix='.png', delete=False) as temp_file:
   molbild=temp_file.name
   img.save(molbild)  # save image



In [6]:
# read DCodes from properties

for atom in mol_mit_Code.GetAtoms():
    if atom.HasProp('DCode'):
        codestring=atom.GetProp('DCode')
        index=atom.GetIdx()+1
        verschiebung, trefferzahl, kuerzungen=calcshift(df,codestring)

## output as table

# initialise list
table_data = []

# collect data
for atom in mol_mit_Code.GetAtoms():
    if atom.HasProp('DCode'):
        codestring = atom.GetProp('DCode')
        index = atom.GetIdx() + 1
        verschiebung, trefferzahl, kuerzungen = calcshift(df, codestring)
        
        # insert data into table
        
        table_data.append({
            "Atom": index,
            "Shift [ppm]": f"{verschiebung:.2f}",
            "Hits": trefferzahl,
            "Shorts": kuerzungen - 1 if (kuerzungen - 1) != 0 else 0
        })

# create Pandas DataFrame
df_table = pd.DataFrame(table_data)

# show image and table
#display(Image(filename=molbild)) #show image

#display(HTML(df_table.to_html(escape=False, index=False)))

molbild_filename = os.path.basename(molbild)

# HTML-Template 
html_content = f"""
<div style="display: flex; align-items: flex-start;">
    <!-- Bild -->
    <div style="margin-right: 20px;">
        <img src="{molbild_filename}" alt="Molbild" style="width:450px;height:auto;">
    </div>
    
    <!-- Tabelle -->
    <div>
        {df_table.to_html(escape=False, index=False)}
    </div>
</div>
"""

# show HTML
display(HTML(html_content))

Atom,Shift [ppm],Hits,Shorts
2,73.60,1,11
3,34.60,1,8
4,44.25,2,11
5,216.50,3,12
7,46.50,15,11
8,44.10,33,12
9,78.80,1,10


## Please wait for finishing output

In [16]:
## cleanup procedure
# start after output is visible
## remove temporary picture

os.remove(molbild)